In [1]:
from __future__ import division
%matplotlib inline
import numpy as np,  matplotlib.pyplot as plt, sys, gc
from time import time
from pylab import plot, show, legend
sys.path.append('/home/postdoc/dsen/Desktop/alphaSMC')
from PF import *
sys.path.append('/home/postdoc/dsen/Desktop/alphaSMC/OU_model')
from OU import *
plt.rcParams['figure.figsize'] = (15.0, 3.0)

In [2]:
y = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/y_OU.npy")
x_0 = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/x_0_OU.npy")
theta = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/theta_OU.npy")
data = dict(x_0=x_0, y=y)
def test_fn(x) : return x
propagate, potential = propagate_OU, potential_OU

In [4]:
N_list = np.arange(2,11)*1000
C_list = 100*np.arange(1,6)+1

start_overall = time()
for (i,N) in enumerate(N_list) :
    for (j,C) in enumerate(C_list) : 
        alpha = d_regular_graph(N, C)
        start = time()
        log_NC_aa1, test_fn_est_aa1 = alphaSMC(data, theta, potential, propagate, test_fn, alpha, 
                                               permute_alpha=True, show_progress=False)
        print("[N,C] =", [N,C], "in ", round(time() - start, 1), "seconds")
print("Total time = ",  round((time()-start_overall)/60, 1), "minutes")

[N,C] =  [2000, 101]  in  8.0 seconds
[N,C] =  [2000, 201]  in  12.8 seconds
[N,C] =  [2000, 301]  in  17.6 seconds
[N,C] =  [2000, 401]  in  22.5 seconds
[N,C] =  [2000, 501]  in  26.2 seconds
[N,C] =  [3000, 101]  in  13.9 seconds
[N,C] =  [3000, 201]  in  21.1 seconds
[N,C] =  [3000, 301]  in  28.0 seconds
[N,C] =  [3000, 401]  in  35.0 seconds
[N,C] =  [3000, 501]  in  41.6 seconds
[N,C] =  [4000, 101]  in  20.7 seconds
[N,C] =  [4000, 201]  in  30.5 seconds
[N,C] =  [4000, 301]  in  39.7 seconds
[N,C] =  [4000, 401]  in  48.6 seconds
[N,C] =  [4000, 501]  in  58.4 seconds
[N,C] =  [5000, 101]  in  28.7 seconds
[N,C] =  [5000, 201]  in  41.0 seconds
[N,C] =  [5000, 301]  in  52.6 seconds
[N,C] =  [5000, 401]  in  64.7 seconds
[N,C] =  [5000, 501]  in  75.7 seconds
[N,C] =  [6000, 101]  in  37.5 seconds
[N,C] =  [6000, 201]  in  52.2 seconds
[N,C] =  [6000, 301]  in  67.0 seconds
[N,C] =  [6000, 401]  in  81.7 seconds
[N,C] =  [6000, 501]  in  95.2 seconds
[N,C] =  [7000, 101]  in  

In [ ]:
N_list = np.arange(2,11)*1000
C_list = 100*np.arange(1,6)+1
rep = 10
logNC_bpf, testfn_bpf = np.zeros((rep,len(N_list))), np.zeros((rep,len(N_list)))
logNC_alpha, testfn_alpha = np.zeros((rep,len(N_list),len(C_list))), np.zeros((rep,len(N_list),len(C_list)))

start_overall = time()
for r in range(rep) :
    for (i,N) in enumerate(N_list) :
        #Bootstrap PF:
        logNC, testfn, _ = bootstrap_PF(data, theta, potential, propagate, test_fn, N, store_paths=False)
        logNC_bpf[r,i], testfn_bpf[r,i] = logNC[-1], testfn[-1]
        del logNC, testfn, _
        gc.collect()
        #alpha SMC:
        for (j,C) in enumerate(C_list) :
            alpha = d_regular_graph(N, C)
            start = time()
            logNC, testfn = alphaSMC(data, theta, potential, propagate, test_fn, alpha, 
                                     permute_alpha=True, show_progress=False)
            logNC_alpha[r,i,j], testfn_alpha[r,i,j] = logNC[-1], testfn[-1] 
            print("[rep,N,C] =", [r+1,N,C], "in", round(time() - start, 1), "seconds")
            del logNC, testfn
            gc.collect()

[rep,N,C] = [1, 2000, 101] in 8.1 seconds
[rep,N,C] = [1, 2000, 201] in 15.6 seconds
[rep,N,C] = [1, 2000, 301] in 41.0 seconds
[rep,N,C] = [1, 2000, 401] in 61.2 seconds
[rep,N,C] = [1, 2000, 501] in 75.7 seconds
[rep,N,C] = [1, 3000, 101] in 42.4 seconds
[rep,N,C] = [1, 3000, 201] in 63.1 seconds
[rep,N,C] = [1, 3000, 301] in 81.0 seconds
[rep,N,C] = [1, 3000, 401] in 92.9 seconds
[rep,N,C] = [1, 3000, 501] in 119.5 seconds
[rep,N,C] = [1, 4000, 101] in 67.0 seconds
[rep,N,C] = [1, 4000, 201] in 78.0 seconds
[rep,N,C] = [1, 4000, 301] in 88.1 seconds
[rep,N,C] = [1, 4000, 401] in 108.0 seconds
[rep,N,C] = [1, 4000, 501] in 139.2 seconds
[rep,N,C] = [1, 5000, 101] in 92.1 seconds
[rep,N,C] = [1, 5000, 201] in 129.8 seconds
[rep,N,C] = [1, 5000, 301] in 163.2 seconds
[rep,N,C] = [1, 5000, 401] in 190.7 seconds
[rep,N,C] = [1, 5000, 501] in 226.2 seconds
[rep,N,C] = [1, 6000, 101] in 124.3 seconds
[rep,N,C] = [1, 6000, 201] in 172.6 seconds
[rep,N,C] = [1, 6000, 301] in 196.2 seconds
[r

In [ ]:
i = 1
np.save("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/logNC_bpf"+str(i), logNC_bpf)
np.save("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/logNC_alpha"+str(i), logNC_alpha)
np.save("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/testfn_bpf"+str(i), testfn_bpf)
np.save("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/testfn_alpha"+str(i), testfn_alpha)

In [11]:
i = 1
logNC_bpf = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/logNC_bpf"+str(i)+".npy")
logNC_alpha = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/logNC_alpha"+str(i)+".npy")
# logNC_airpf = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/logNC_airpf"+str(i)+".npy")

testfn_bpf = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/testfn_bpf"+str(i)+".npy")
testfn_alpha = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/testfn_alpha"+str(i)+".npy")
# testfn_airpf = np.load("/home/postdoc/dsen/Desktop/alphaSMC/data/data_OU/testfn_airpf"+str(i)+".npy")

In [12]:
np.shape(testfn_airpf)

(3, 10)

In [ ]:
plt.rcParams['figure.figsize'] = (5.0, 4.0)
fig, ax = plt.subplots()
ax.set_xscale('log', basex=2)
ax.plot(N_list, np.sqrt(N_list)*np.std(logNC,1), "o-", label="empirical variance")
# ax.plot(N_list, 10/np.sqrt(N_list), label="Theory")
plt.grid("True")
plt.legend()
plt.xlabel("N")
plt.ylabel("Variance");